# Bluetooth examples and tests with MicroPython and ESP32
So far the hadn't been support for bluetooth in MicroPython. Kinda just came out and here you can find some tests an examples based on the following documentation: [uBLE MicroPython](https://docs.micropython.org/en/latest/library/ubluetooth.html)

In [1]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


First `ubluetooth` is imported and then `BLE` is defined. Some constants are defined in order to handle the interrupts from BLE. Here every of them is defined, however for a more specific use only the needed ones must be there.

In [2]:
import ubluetooth
import utime
from micropython import const

_IRQ_CENTRAL_CONNECT                 = const(1 << 0)
_IRQ_CENTRAL_DISCONNECT              = const(1 << 1)
_IRQ_GATTS_WRITE                     = const(1 << 2)
_IRQ_GATTS_READ_REQUEST              = const(1 << 3)
_IRQ_SCAN_RESULT                     = const(1 << 4)
_IRQ_SCAN_COMPLETE                   = const(1 << 5)
_IRQ_PERIPHERAL_CONNECT              = const(1 << 6)
_IRQ_PERIPHERAL_DISCONNECT           = const(1 << 7)
_IRQ_GATTC_SERVICE_RESULT            = const(1 << 8)
_IRQ_GATTC_CHARACTERISTIC_RESULT     = const(1 << 9)
_IRQ_GATTC_DESCRIPTOR_RESULT         = const(1 << 10)
_IRQ_GATTC_READ_RESULT               = const(1 << 11)
_IRQ_GATTC_WRITE_STATUS              = const(1 << 12)
_IRQ_GATTC_NOTIFY                    = const(1 << 13)
_IRQ_GATTC_INDICATE                  = const(1 << 14)

ble=ubluetooth.BLE()

Afterwards we must activate bluetooth (if it is going to be used), in order to use any other coming method.

In [3]:
if ble.active()== False:
    ble.active(True)

Then checking if it is active.

In [4]:
print(ble.active())

True


MAC is printed just in case.

In [5]:
print('BLE current mac address ', ble.config('mac'))

BLE current mac address  b'\xd0p\xfe?\xf8C'


These constants are then included in our interrupts handler for every case, so then it is added with `BLE.irq`.

In [6]:
def bt_irq(event, data):
    
    if event == _IRQ_CENTRAL_CONNECT:
        # A central has connected to this peripheral.
        conn_handle, addr_type, addr = data
    elif event == _IRQ_CENTRAL_DISCONNECT:
        # A central has disconnected from this peripheral.
        conn_handle, addr_type, addr = data
    elif event == _IRQ_GATTS_WRITE:
        # A central has written to this characteristic or descriptor.
        conn_handle, attr_handle = data
    elif event == _IRQ_GATTS_READ_REQUEST:
        # A central has issued a read. Note: this is a hard IRQ.
        # Return None to deny the read.
        # Note: This event is not supported on ESP32.
        conn_handle, attr_handle = data
    elif event == _IRQ_SCAN_RESULT:
        # A single scan result.
        addr_type, addr, connectable, rssi, adv_data = data
        print(data)
    elif event == _IRQ_SCAN_COMPLETE:
        # Scan duration finished or manually stopped.
        pass
    elif event == _IRQ_PERIPHERAL_CONNECT:
        # A successful gap_connect().
        conn_handle, addr_type, addr = data
    elif event == _IRQ_PERIPHERAL_DISCONNECT:
        # Connected peripheral has disconnected.
        conn_handle, addr_type, addr = data
    elif event == _IRQ_GATTC_SERVICE_RESULT:
        # Called for each service found by gattc_discover_services().
        conn_handle, start_handle, end_handle, uuid = data
    elif event == _IRQ_GATTC_CHARACTERISTIC_RESULT:
        # Called for each characteristic found by gattc_discover_services().
        conn_handle, def_handle, value_handle, properties, uuid = data
    elif event == _IRQ_GATTC_DESCRIPTOR_RESULT:
        # Called for each descriptor found by gattc_discover_descriptors().
        conn_handle, dsc_handle, uuid = data
    elif event == _IRQ_GATTC_READ_RESULT:
        # A gattc_read() has completed.
        conn_handle, value_handle, char_data = data
    elif event == _IRQ_GATTC_WRITE_STATUS:
        # A gattc_write() has completed.
        conn_handle, value_handle, status = data
    elif event == _IRQ_GATTC_NOTIFY:
        # A peripheral has sent a notify request.
        conn_handle, value_handle, notify_data = data
    elif event == _IRQ_GATTC_INDICATE:
        # A peripheral has sent an indicate request.
        conn_handle, value_handle, notify_data = data
        
ble.irq(bt_irq)

Finally scanning available BLE devices. Here the whole code is added so that it works.

In [7]:
import ubluetooth
import utime
from micropython import const
_IRQ_CENTRAL_CONNECT                 = const(1 << 0)
_IRQ_CENTRAL_DISCONNECT              = const(1 << 1)
_IRQ_GATTS_WRITE                     = const(1 << 2)
_IRQ_GATTS_READ_REQUEST              = const(1 << 3)
_IRQ_SCAN_RESULT                     = const(1 << 4)
_IRQ_SCAN_COMPLETE                   = const(1 << 5)
_IRQ_PERIPHERAL_CONNECT              = const(1 << 6)
_IRQ_PERIPHERAL_DISCONNECT           = const(1 << 7)
_IRQ_GATTC_SERVICE_RESULT            = const(1 << 8)
_IRQ_GATTC_CHARACTERISTIC_RESULT     = const(1 << 9)
_IRQ_GATTC_DESCRIPTOR_RESULT         = const(1 << 10)
_IRQ_GATTC_READ_RESULT               = const(1 << 11)
_IRQ_GATTC_WRITE_STATUS              = const(1 << 12)
_IRQ_GATTC_NOTIFY                    = const(1 << 13)
_IRQ_GATTC_INDICATE                  = const(1 << 14)

ble=ubluetooth.BLE()

if ble.active()== False:
    ble.active(True)
    
print(ble.active())

print('BLE current mac address ', ble.config('mac'))


def bt_irq(event, data):
    if event == _IRQ_CENTRAL_CONNECT:
        # A central has connected to this peripheral.
        conn_handle, addr_type, addr = data
    elif event == _IRQ_CENTRAL_DISCONNECT:
        # A central has disconnected from this peripheral.
        conn_handle, addr_type, addr = data
    elif event == _IRQ_GATTS_WRITE:
        # A central has written to this characteristic or descriptor.
        conn_handle, attr_handle = data
    elif event == _IRQ_GATTS_READ_REQUEST:
        # A central has issued a read. Note: this is a hard IRQ.
        # Return None to deny the read.
        # Note: This event is not supported on ESP32.
        conn_handle, attr_handle = data
    elif event == _IRQ_SCAN_RESULT:
        # A single scan result.
        addr_type, addr, connectable, rssi, adv_data = data
        print(data)
    elif event == _IRQ_SCAN_COMPLETE:
        # Scan duration finished or manually stopped.
        pass
    elif event == _IRQ_PERIPHERAL_CONNECT:
        # A successful gap_connect().
        conn_handle, addr_type, addr = data
    elif event == _IRQ_PERIPHERAL_DISCONNECT:
        # Connected peripheral has disconnected.
        conn_handle, addr_type, addr = data
    elif event == _IRQ_GATTC_SERVICE_RESULT:
        # Called for each service found by gattc_discover_services().
        conn_handle, start_handle, end_handle, uuid = data
    elif event == _IRQ_GATTC_CHARACTERISTIC_RESULT:
        # Called for each characteristic found by gattc_discover_services().
        conn_handle, def_handle, value_handle, properties, uuid = data
    elif event == _IRQ_GATTC_DESCRIPTOR_RESULT:
        # Called for each descriptor found by gattc_discover_descriptors().
        conn_handle, dsc_handle, uuid = data
    elif event == _IRQ_GATTC_READ_RESULT:
        # A gattc_read() has completed.
        conn_handle, value_handle, char_data = data
    elif event == _IRQ_GATTC_WRITE_STATUS:
        # A gattc_write() has completed.
        conn_handle, value_handle, status = data
    elif event == _IRQ_GATTC_NOTIFY:
        # A peripheral has sent a notify request.
        conn_handle, value_handle, notify_data = data
    elif event == _IRQ_GATTC_INDICATE:
        # A peripheral has sent an indicate request.
        conn_handle, value_handle, notify_data = data
        
ble.irq(bt_irq)

ble.gap_scan(10000, 30000, 30000)
ini=utime.ticks_ms()
while utime.ticks_ms()-ini<=10000:
    pass

True
BLE current mac address  b'\x01\x00\x00\x00\x01\x00'
(0, b'\x84\r\x8e\x17\x88\x86', True, -47, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -51, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -54, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -47, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -50, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -54, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -54, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -47, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -51, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -54, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -47, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -50, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -54, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -48, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -50, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -55, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -47, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -50, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -54, b'')
(0, b'\x84\r\x8e\x17\x88\x86', True, -47, b'')
(0

Traceback (most recent call last):
  File "<stdin>", line 85, in <module>
KeyboardInterrupt: 
